##### 问题:
我们想使用标准的比较操作符（如>=、!=、<=等）在类实例之间进行比较，但是又不
想编写大量的特殊方法。

##### 解决方案:
通过为每种比较操作符实现一个特殊方法，Python 中的类可以支持比较操作。例如，
要支持>=操作符，可以在类中定义一个__ge__()方法。虽然只定义一个方法不算什
么，但如果要实现每种可能的比较操作，那么实现这么多特殊方法则很快会变得
繁琐。<li><b>ge代表英文Greater than or equal to.
<li><b>le代表英文Less than or equal to.

functools.total_ordering 装饰器可用来简化这个过程。要使用它，可以用它来装饰一个类，
然后定义__eq__()以及另一个比较方法（\_\_lt\_\_、\_\_le\_\_、\_\_gt\_\_或者\_\_ge\_\_）。那么装
饰器就会自动为我们实现其他的比较方法。<li><b>lt:小于,le:小于等于,gt:大于,ge:大于等于,ne:不等于,eq:等于</b></li>

作为示例，让我们来构建一些房子并为其添加一些房间吧，然后根据房子的大小来进
行比较：

In [5]:
from functools import total_ordering  #引入了类装饰器
class Room:
    def __init__(self, name, length, width):
        self.name = name
        self.length = length
        self.width = width
        self.square_feet = self.length * self.width
@total_ordering
class House:
    def __init__(self, name, style):
        self.name = name
        self.style = style
        self.rooms = list()
    @property
    def living_space_footage(self):
        return sum(r.square_feet for r in self.rooms)
    def add_room(self, room):
        self.rooms.append(room)
    def __str__(self):
        return '{}: {} square foot {}'.format(self.name,self.living_space_footage,self.style)
    def __eq__(self, other):
        return self.living_space_footage == other.living_space_footage
    def __lt__(self, other):
        return self.living_space_footage < other.living_space_footage




这里，House 类已经用@total_ordering 来进行装饰了。我们定义了__eq__()和__lt__()来
根据房间的总面积对房子进行比较。只需要定义这两个特殊方法就能让其他所有的比
较操作正常工作。示例如下：

In [6]:
# Build a few houses, and add rooms to them
h1 = House('h1', 'Cape')
h1.add_room(Room('Master Bedroom', 14, 21))
h1.add_room(Room('Living Room', 18, 20))
h1.add_room(Room('Kitchen', 12, 16))
h1.add_room(Room('Office', 12, 12))
h2 = House('h2', 'Ranch')
h2.add_room(Room('Master Bedroom', 14, 21))
h2.add_room(Room('Living Room', 18, 20))
h2.add_room(Room('Kitchen', 12, 16))
h3 = House('h3', 'Split')
h3.add_room(Room('Master Bedroom', 14, 21))
h3.add_room(Room('Living Room', 18, 20))
h3.add_room(Room('Office', 12, 16))
h3.add_room(Room('Kitchen', 15, 17))
houses = [h1, h2, h3]
print('Is h1 bigger than h2?', h1 > h2) # prints True
print('Is h2 smaller than h3?', h2 < h3) # prints True
print('Is h2 greater than or equal to h1?', h2 >= h1) # Prints False
print('Which one is biggest?', max(houses)) # Prints 'h3: 1101-square-foot Split'
print('Which is smallest?', min(houses)) # Prints 'h2: 846-square-foot Ranch'

Is h1 bigger than h2? True
Is h2 smaller than h3? True
Is h2 greater than or equal to h1? False
Which one is biggest? h3: 1101 square foot Split
Which is smallest? h2: 846 square foot Ranch


如果我们曾经编写过代码让类支持所有的基本比较操作符，那么装饰器 total_ordering
对我们而言就并非那么神奇：它从字面上定义了从每个比较方法到其他所有需要该方
法的映射关系。因此，如果在类中定义了__lt__()，那么就会利用它来构建其他所有的
比较操作符。实际上就是在类中填充以下方法：


In [7]:
class House:
    def __eq__(self, other):    
        pass
    def __lt__(self, other):
        pass
 # Methods created by @total_ordering    #

    __le__ = lambda self, other: self < other or self == other
    __gt__ = lambda self, other: not (self < other or self == other)
    __ge__ = lambda self, other: not (self < other)
    __ne__ = lambda self, other: not self == other 

的确，自行编写这些方法并不难，但@total_ordering 让这一过程变得更加简单了。